### Import module we need

In [2]:
import pydicom

### Import Data

In [37]:
ct_slice_0 = pydicom.dcmread('Input/Task3/000000.dcm')
ct_slice_1 = pydicom.dcmread('Input/Task3/000001.dcm')
print(ct_slice_0)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 226
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.9590.100.1.2.271988646011540584515081711502032149853
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.9590.100.1.3.100.9.4
(0002, 0013) Implementation Version Name         SH: 'MATLAB IPT 9.4'
(0002, 0016) Source Application Entity Title     AE: 'MAROTECH'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'GSI MONO']
(0008, 0012) Instance Creation Date              DA: '20180427'
(0008, 0013) Instance Creation Time              TM: '102217'
(

### Task 3.1 
- Load DICOM files in the folder, ‘blind’. Each DICOM file name represents CT slice (transverse) index . Thus, each file consists of metadata and image data of the corresponding slice . From metadata, check image attributes such as ‘pixelSpacing’ and ‘SpacingBetweenSlices’. Construct 3D data . From 3D data, pick some transverse, sagittal and coronal slices, and display each slice image with axis scales in cm.   

3.1.1. From metadata, check image attributes such as ‘pixelSpacing’ and ‘SpacingBetweenSlices’
- 추후 사용할 수 있으므로 한번에 뽑아서 DICT 형식으로 놔두면 좋을 듯

In [38]:
ct_slice_0.dir()[:5]

['AccessionNumber',
 'AcquisitionDate',
 'AcquisitionNumber',
 'AcquisitionTime',
 'AdditionalPatientHistory']

In [39]:
ct_slice_0.AccessionNumber

'1800412745'

In [40]:
getattr(ct_slice_0, 'AccessionNumber')

'1800412745'

In [41]:
meta_data_atr_l = ct_slice_0.dir()
meta_data_dict = dict(zip(meta_data_atr_l, [getattr(ct_slice_0, atr) for atr in meta_data_atr_l]))
meta_data_dict

{'AccessionNumber': '1800412745',
 'AcquisitionDate': '20180427',
 'AcquisitionNumber': '1',
 'AcquisitionTime': '102131.507',
 'AdditionalPatientHistory': '',
 'BitsAllocated': 16,
 'BitsStored': 16,
 'BodyPartExamined': 'UNKNOWN',
 'Columns': 512,
 'ContentDate': '20180427',
 'ContentTime': '102217',
 'ConvolutionKernel': 'STANDARD',
 'DataCollectionDiameter': '500.0',
 'DistanceSourceToDetector': '1097.61',
 'DistanceSourceToPatient': '625.61',
 'Exposure': '27',
 'ExposureTime': '456',
 'FilterType': 'BODY FILTER',
 'FocalSpots': '1.2',
 'FrameOfReferenceUID': '1.2.840.113619.2.416.239305910505007702453072198152204460922',
 'GantryDetectorTilt': '0.0',
 'GeneratorPower': '63000',
 'HighBit': 15,
 'ImageOrientationPatient': [1, 0, 0, 0, 1, 0],
 'ImagePositionPatient': [-208.104, -213, -80],
 'ImageType': ['ORIGINAL', 'PRIMARY', 'AXIAL', 'GSI MONO'],
 'InstanceCreationDate': '20180427',
 'InstanceCreationTime': '102217',
 'InstanceNumber': '1',
 'InstitutionName': 'PNUYH',
 'KVP': '1

In [42]:
meta_data_dict.keys()

dict_keys(['AccessionNumber', 'AcquisitionDate', 'AcquisitionNumber', 'AcquisitionTime', 'AdditionalPatientHistory', 'BitsAllocated', 'BitsStored', 'BodyPartExamined', 'Columns', 'ContentDate', 'ContentTime', 'ConvolutionKernel', 'DataCollectionDiameter', 'DistanceSourceToDetector', 'DistanceSourceToPatient', 'Exposure', 'ExposureTime', 'FilterType', 'FocalSpots', 'FrameOfReferenceUID', 'GantryDetectorTilt', 'GeneratorPower', 'HighBit', 'ImageOrientationPatient', 'ImagePositionPatient', 'ImageType', 'InstanceCreationDate', 'InstanceCreationTime', 'InstanceNumber', 'InstitutionName', 'KVP', 'LargestImagePixelValue', 'Manufacturer', 'ManufacturerModelName', 'Modality', 'NameOfPhysiciansReadingStudy', 'OperatorsName', 'OtherPatientIDs', 'PatientAge', 'PatientBirthDate', 'PatientID', 'PatientName', 'PatientPosition', 'PatientSex', 'PerformedProcedureStepDescription', 'PerformedProcedureStepID', 'PerformedProcedureStepStartDate', 'PerformedProcedureStepStartTime', 'PerformingPhysicianName',

- test: pixeldSpacing and SpacingBetweenSlices

In [43]:
print(f"pixelSpacing: {meta_data_dict.pixelSpacing}")

AttributeError: 'dict' object has no attribute 'pixelSpacing'

In [44]:
print(f"pixelSpacing: {meta_data_dict["PixelSpacing"]}")

pixelSpacing: [0.832031, 0.832031]


In [45]:
print(f"SpacingBetweenSlices: {meta_data_dict["SpacingBetweenSlices"]}")

KeyError: 'SpacingBetweenSlices'

In [47]:
ct_slice_0.SpacingBetweenSlices

AttributeError: 'FileDataset' object has no attribute 'SpacingBetweenSlices'

In [48]:
ct_slice_1.SpacingBetweenSlices

AttributeError: 'FileDataset' object has no attribute 'SpacingBetweenSlices'